In [1]:
import tensorflow as tf
import numpy as np
import random
import datetime
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import scipy.sparse as sp
import importlib
import GNN
importlib.reload(GNN)

<module 'GNN' from 'C:\\Users\\BBD\\Desktop\\图计算\\图神经网络\\GNN.py'>

In [2]:
A, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = GNN.load_data_v1('cora')
features = features.toarray()

Dataset has 2708 nodes, 5429 edges, 1433 features.


### 模型存储演示

In [3]:
model = GNN.GNN(layers=[16,16],batch_size=1024,epoch=150,learning_rate=0.05,dropout_rate=0.5,l2_reg=1e-4)
model.fit(features,y_train,train_mask,GNN.get_A_GCN(A))




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
2019-12/03/19 16:21:11	epoch 0	13.583666
2019-12/03/19 16:21:11	epoch 1	11.170485
2019-12/03/19 16:21:12	epoch 2	8.4724045
2019-12/03/19 16:21:12	epoch 3	6.53085
2019-12/03/19 16:21:12	epoch 4	5.3906484
2019-12/03/19 16:21:12	epoch 5	4.756566
2019-12/03/19 16:21:13	epoch 6	4.4235644
2019-12/03/19 16:21:13	epoch 7	4.2573285
2019-12/03/19 16:21:13	epoch 8	4.16395
2019-12/03/19 16:21:13	epoch 9	4.0964017
2019-12/03/19 16:21:14	epoch 10	4.0431466
2019-12/03/19 16:21:14	epoch 11	3.9951315
2019-12/03/19 16:21:14	epoch 12	3.9576674
2019-12/03/19 16:21:14	epoch 13	3.9115052
2019-12/03/19 16:21:14	epoch 14	3.8696601
2019-12/03/19 16:21:15	epoch 15	3.82964
2019-12/03/19 16:21:15	epoch 16	3.7922566
2019-12/03/19 16:21:15	epoch 17	3.7554097
2019-12/03/19 16:21:15	epoch 18	3.7213097
2019-12/03/19 16:21:16	epoch 19	3.6894436
2019-12/03/19 16:21:16	epoch 20	3.65737
2019-12/03/19 16:21:16	epoch 21	3.624554

In [4]:
model.save_parameters('./model/demo_model')

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

./model/demo_model


In [5]:
model2 = GNN.GNN()
model2.load_parameters('./model/demo_model')

INFO:tensorflow:Restoring parameters from ./model/demo_model


In [6]:
y_train_predict = model2.predict_proba(features,GNN.get_A_GCN(A))
roc_auc_score(y_train[train_mask], y_train_predict[train_mask],average='micro')

0.9996598639455783

### 拟合GCN

In [7]:
model = GNN.GNN(layers=[128,128],batch_size=512,epoch=150,
                learning_rate=0.05,dropout_rate=0.5,l2_reg=1e-4,
                activation = tf.nn.tanh)
model.fit(features,y_train,train_mask,GNN.get_A_GCN(A))

2019-12/03/19 16:21:50	epoch 0	10.366122
2019-12/03/19 16:21:50	epoch 1	7.39401
2019-12/03/19 16:21:51	epoch 2	6.37437
2019-12/03/19 16:21:51	epoch 3	5.4172416
2019-12/03/19 16:21:51	epoch 4	4.491373
2019-12/03/19 16:21:51	epoch 5	3.8577666
2019-12/03/19 16:21:52	epoch 6	3.5565245
2019-12/03/19 16:21:52	epoch 7	3.427588
2019-12/03/19 16:21:52	epoch 8	3.484819
2019-12/03/19 16:21:53	epoch 9	3.5476918
2019-12/03/19 16:21:53	epoch 10	3.4393754
2019-12/03/19 16:21:53	epoch 11	3.2762985
2019-12/03/19 16:21:54	epoch 12	3.090858
2019-12/03/19 16:21:54	epoch 13	3.001508
2019-12/03/19 16:21:54	epoch 14	3.002017
2019-12/03/19 16:21:54	epoch 15	2.9969835
2019-12/03/19 16:21:55	epoch 16	2.9742227
2019-12/03/19 16:21:55	epoch 17	2.9408584
2019-12/03/19 16:21:55	epoch 18	2.9107938
2019-12/03/19 16:21:55	epoch 19	2.8859913
2019-12/03/19 16:21:56	epoch 20	2.8635738
2019-12/03/19 16:21:56	epoch 21	2.8461628
2019-12/03/19 16:21:56	epoch 22	2.8368092
2019-12/03/19 16:21:57	epoch 23	2.828196
2019-12/03/19

训练精度

In [8]:
y_train_predict = model.predict_proba(features,GNN.get_A_GCN(A))
roc_auc_score(y_train[train_mask], y_train_predict[train_mask],average='micro')

1.0

Validation精度

In [9]:
y_val_predict = model.predict_proba(features,GNN.get_A_GCN(A))
roc_auc_score(y_val[val_mask], y_val_predict[val_mask],average='micro')

0.9342896666666666

测试精度

In [10]:
y_test_predict = model.predict_proba(features,GNN.get_A_GCN(A))
roc_auc_score(y_test[test_mask], y_test_predict[test_mask],average='micro')

0.9492335833333334

### 拟合普通神经网络

In [11]:
model = GNN.GNN(layers=[128,128],batch_size=512,epoch=150,
                learning_rate=0.05,dropout_rate=0.5,l2_reg=1e-4,
                activation = tf.nn.tanh)
model.fit(features,y_train,train_mask)

2019-12/03/19 16:22:31	epoch 0	17.05971
2019-12/03/19 16:22:31	epoch 1	15.010021
2019-12/03/19 16:22:31	epoch 2	13.264672
2019-12/03/19 16:22:32	epoch 3	11.848945
2019-12/03/19 16:22:32	epoch 4	11.017592
2019-12/03/19 16:22:32	epoch 5	10.135329
2019-12/03/19 16:22:32	epoch 6	9.3574915
2019-12/03/19 16:22:33	epoch 7	8.599147
2019-12/03/19 16:22:33	epoch 8	7.950401
2019-12/03/19 16:22:33	epoch 9	7.394907
2019-12/03/19 16:22:33	epoch 10	6.8600135
2019-12/03/19 16:22:34	epoch 11	6.316213
2019-12/03/19 16:22:34	epoch 12	5.995195
2019-12/03/19 16:22:34	epoch 13	5.6792226
2019-12/03/19 16:22:34	epoch 14	5.3815813
2019-12/03/19 16:22:35	epoch 15	5.025161
2019-12/03/19 16:22:35	epoch 16	4.595568
2019-12/03/19 16:22:35	epoch 17	4.248346
2019-12/03/19 16:22:35	epoch 18	3.9443119
2019-12/03/19 16:22:36	epoch 19	3.6337862
2019-12/03/19 16:22:36	epoch 20	3.3414135
2019-12/03/19 16:22:36	epoch 21	3.1474857
2019-12/03/19 16:22:36	epoch 22	3.0605807
2019-12/03/19 16:22:37	epoch 23	2.9960458
2019-12/03/

训练精度

In [12]:
y_train_predict = model.predict_proba(features)
roc_auc_score(y_train[train_mask], y_train_predict[train_mask],average='micro')

1.0

Validation精度

In [13]:
y_val_predict = model.predict_proba(features)
roc_auc_score(y_val[val_mask], y_val_predict[val_mask],average='micro')

0.7926506666666667

测试精度

In [14]:
y_test_predict = model.predict_proba(features)
roc_auc_score(y_test[test_mask], y_test_predict[test_mask],average='micro')

0.7827316666666667